In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp
import copy
from EVFleet import * 
import pandas as pd

In [3]:
def create_energy_set(start, M, H, pv, wt):
    days = 365 // M -1
    q = np.zeros((M, H))
    Q = np.zeros((M, days * 24))
    for m in range(M):
        start_idx = m * days * 24
        end_idx = (m + 1) * days * 24
        Q[m, :] = pv[start_idx:end_idx] + wt[start_idx:end_idx]
        q[m, :] = Q[m, start:start+H]

    return q

In [5]:
DAYS = 1
H=24
N=10
M=1
start = 10

# Set seed for reproducibility
np.random.seed(42)

hours = np.arange(start, start + 24)
base_price = 0.15
amplitude = 0.10

noise = np.random.normal(0, 0.01, size=24*DAYS)
# Symmetric price
prices = base_price + amplitude * (
    np.sin((hours - 8) / 24 * 2 * np.pi) + np.sin((hours - 18) / 24 * 2 * np.pi)
) + noise
prices = np.clip(prices, 0.05, None)
    
# Add asymmetric buy/sell prices
margin = 0.03
c = prices + margin
p = prices - margin
p = np.clip(p, 0.01, None)  # Prevent negative prices

# Load CSV data
pv_energy = '2023_photovoltaic_total_at.csv'  # Replace with your actual file path
wt_energy = '2023_windpower_total_at.csv'  # Replace with your actual file path

pv = pd.read_csv(pv_energy, delimiter=';')
wt = pd.read_csv(wt_energy, delimiter=';')

pv_arr = pv['Power (MW)'].to_numpy()
wt_arr = wt['Power (MW)'].to_numpy()


q = create_energy_set(start, M, H*DAYS, pv_arr, wt_arr) / 100

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 12.5))  # 1 row, 2 columns

axs[0].set_title(f'RES energy generation for {H*DAYS} hours for {M} microgrids')
for m in range(M): 
    axs[0].plot(np.arange(start, start+H*DAYS), q[m,:], label=f"pv{m}")
axs[0].legend()
axs[0].set_xlabel('Time, hour')
axs[0].set_ylabel('Power, MW')
axs[0].grid(True)
# First subplot: Buy and Sell Prices
axs[1].plot(hours, c, label='Buy Price (€/kWh)')
axs[1].plot(hours, p, label='Sell Price (€/kWh)')
axs[1].plot(hours, prices, label='Base Market Price (€/kWh)', linestyle='--', alpha=0.7)
axs[1].set_title("Simulated 24-Hour Electricity Buy and Sell Prices")
axs[1].set_xlabel("Hour")
axs[1].set_ylabel("Price (€/kWh)")
axs[1].grid(axis='y')
axs[1].legend()

plt.tight_layout()
plt.savefig("Decentralised with data environmental parameters", dpi=300)
plt.show()

In [6]:
fleet = EVFleet(xlsx_path='EV_UsageProfile.xlsx', num_vehicles=N*M)
results = fleet.simulate_multiple_days(num_days=1)

L_usage = np.zeros((M,N,DAYS+1,4), dtype='int')
for row in results:
    L_usage[row['EV'] // N, row['EV'] % N, row['Day']-1, 0] = int(row['Initial SoC']) / 100
    L_usage[row['EV'] // N, row['EV'] % N, row['Day']-1, 1] = int(row['Goal (kWh)']) / 100
    L_usage[row['EV'] // N, row['EV'] % N, row['Day']-1, 2] = int(row['Arrival (h)'])
    L_usage[row['EV'] // N, row['EV'] % N, row['Day']-1, 3] = int(row['Departure (h)']) + 24 if int(row['Departure (h)']) + 24 < 34 else 33

    print(row['EV'] // N, row['EV'] % N, L_usage[row['EV'] // N, row['EV'] % N, row['Day']-1, :])

0 0 [18 80 16 30]
0 1 [18 80 14 31]
0 2 [18 80 13 30]
0 3 [15 80 17 31]
0 4 [17 40 11 33]
0 5 [10 80 17 31]
0 6 [18 80 16 30]
0 7 [15 80 16 30]
0 8 [16 80 16 30]
0 9 [18 80 14 31]


In [9]:
fig, axs = plt.subplots(2, 1, figsize=(10,12.5))
for m in range(M):
    for n in range(N):
        L_init, L_target, L_arrival, L_dep = L_usage[m,n,0,:]
        y = [L_init, L_target]
        x = [L_arrival, L_dep]
        axs[m].plot(x,y)
        axs[m].grid(True)
axs[0].set_title("Charging plans for each microgrid");
plt.tight_layout()
plt.savefig("Charging plans for each microgrid", dpi=300)
plt.show()

L_target = L_usage[:,:,0,1]
L_init = L_usage[:,:,0,0]
L_arrival = L_usage[:,:,0,2]
L_depart = L_usage[:,:,0,3]

In [11]:
# Constants
rho = 10 #1.8
gamma = 0.1 #0.1

epsilon = 0.1
c1 = 5
eta = 0.9

# Random seeds for reproducibility
np.random.seed(42)



L_min = np.ones((N, H)) * 0
L_max = np.ones((N, H)) * 100

d_min = np.zeros((N, H))
d_max = np.ones((N, H)) * 10

f_min = np.zeros((N, H))
f_max = np.ones((N, H)) * 10

l_min = np.zeros((N, H))
l_max = np.ones((N, H)) * 10

m_min = np.zeros((N, H))
m_max = np.ones((N, H)) * 10

# Initial values
x = np.array([np.zeros(5*H) for _ in range(N)])  # d, f, l, m, L[1:]
lambdas = np.zeros(H)

x_prev = copy.deepcopy(x)
iteration = 0

# Collect data for plotting
d_history = np.array([np.zeros(H) for n in range(N)])
f_history = np.array([np.zeros(H) for n in range(N)])
l_history = np.array([np.zeros(H) for n in range(N)])
m_history = np.array([np.zeros(H) for n in range(N)])
L_history = np.array([np.zeros(H) for n in range(N)])

cost_terms = []
penalty_terms = []

H_n_T_x = np.zeros((N,H))
H_n_T_xi = np.zeros((N,H))
# ADMM loop
while True:
    iteration += 1
    
    final_cost = 0
    for n in range(N):
        # Variables
        constraints = []
        xi = cp.Variable(5*H)
        d = xi[:H]
        f = xi[H:2*H]
        l = xi[2*H:3*H]
        m = xi[3*H:4*H]
        L = xi[4*H:]

        start_time = L_arrival[0][n]-start
        end_time = L_depart[0][n]-start

        # Objective components
        net_cost = cp.sum(cp.multiply(c, d) / q[0] - cp.multiply(p, f) / q[0])
    
        charge_penalty = cp.sum_squares(L[end_time] - L_target[0][n])
    
        h_n_T_x = -d + f + l - m
        admm_penalty = (rho / 2) *  cp.sum_squares(h_n_T_x  + cp.sum(H_n_T_x,axis=0) - H_n_T_x[n] - q[0] - 1/rho * lambdas)

        admm_update = 11/2 *cp.sum_squares(xi - x_prev[n]) # 5.5

        objective = cp.Minimize(c1*net_cost + charge_penalty + admm_penalty +  admm_update)

        # Constraints
        constraints += [
            L >= L_min[n],
            L <= L_max[n],
            d >= d_min[n],
            d <= d_max[n],
            f >= f_min[n],
            f <= f_max[n],
            l >= l_min[n],
            l <= l_max[n],
            m >= m_min[n],
            m <= m_max[n]
        ]
        
        constraints += [
             L[L_arrival[0,n]-start] == L_init[0][n] + l[L_arrival[0,n]-start] - m[L_arrival[0,n]-start]
        ]
        for h in range(1, H):
            constraints += [
                L[h] == L[h-1] + l[h] - m[h],
                cp.abs(l[h-1] - l[h]) <= 5,
                cp.abs(m[h-1] - m[h]) <= 5,
                cp.abs(d[h-1] - d[h]) <= 5,
                cp.abs(f[h-1] - f[h]) <= 5,
            ]
            
        for h in range(start, start + H):
                if h >= L_depart[0,n]:
                    constraints += [
                        l[h-start] == 0,
                        m[h-start] == 0,
                        d[h-start] == 0
                    ]
                if h <= L_arrival[0,n]:
                    constraints += [
                        l[h-start] == 0,
                        m[h-start] == 0,
                        d[h-start] == 0
                    ]                    
            

        # Solve
        prob = cp.Problem(objective, constraints)
        solvers = [cp.SCS, cp.ECOS, cp.CLARABEL, cp.OSQP, cp.GUROBI]  
        solution_found = False
    
        for solver in solvers:
            try:
                prob.solve(solver=solver)
                if prob.status in ["optimal"]:
                    solution_found = True
                    break
                else:
                    print(f"Solver {solver} failed. Trying next...")
            except Exception as e:
                print(f"Solver {solver} encountered an error: {e}")
    
        if not solution_found:
            print("No valid solution found with available solvers.")
            
        """
        print(f"######### Parameters for Agent: {n} ##########")
        print(f"Net cost: {net_cost.value}")
        print(f"Charge penalty: {charge_penalty.value}")
        print(f"Lambda penalty: {admm_penalty.value}")
        print(f"Update penalty: {admm_update.value}")
        print(f"Objective value: {objective.value}")
        print(f"############################################")
        print(f" ")
        """
        # Save values
        x[n] = copy.deepcopy(xi.value)
        H_n_T_xi[n] = copy.deepcopy(h_n_T_x.value)

        # Collect data for plotting
        d_history[n] = copy.deepcopy(d.value)
        f_history[n] = copy.deepcopy(f.value)
        l_history[n] = copy.deepcopy(l.value)
        m_history[n] = copy.deepcopy(m.value)
        L_history[n] = copy.deepcopy(L.value)

        final_cost += net_cost.value + charge_penalty.value
    
    H_n_T_x = copy.deepcopy(H_n_T_xi)
    
    lambdas -= gamma * rho * (np.sum(H_n_T_x, axis=0) - q[0])
    #gamma *=0.9
    print("--------------------------------------------")
    print(f"ITERATION: {iteration}, difference: {np.linalg.norm(x-x_prev)}, lambda {np.linalg.norm(np.sum(H_n_T_x, axis=0) - q)}")
    print("--------------------------------------------")
    print(f" ")
    
    if np.linalg.norm(x-x_prev) <= epsilon:
        break
    print(f"Charge penalty: {charge_penalty.value}")
    x_prev = copy.deepcopy(x)
    
print(final_cost)

--------------------------------------------
ITERATION: 1, difference: 142.8662509718654, lambda 85.80934179356882
--------------------------------------------
 
Charge penalty: 5875.3463815889745


KeyboardInterrupt: 

In [ ]:
# Plotting
fig, axes = plt.subplots(5, 1, figsize=(10, 5*2.5), sharex=True)

for n in range(N):
    #axes[0].plot(np.full(H,L_target[n]))
    axes[0].plot(L_history[n], label =f'agent {n}')
    axes[0].grid(axis='y')
    axes[0].legend()
    axes[0].set_ylabel("Load, Wh*h")

for n in range(N):
    axes[1].plot(d_history[n])
    axes[1].grid(axis='y')
    axes[1].set_ylabel("Draw from external grid, Wh")

for n in range(N):
    axes[2].plot(f_history[n])
    axes[2].grid(axis='y')
    axes[2].set_ylabel("Feed to external grid, Wh")
    
for n in range(N):
    axes[3].plot(m_history[n])
    axes[3].grid(axis='y')
    axes[3].set_ylabel("Discharging to local DC-grid, Wh")
    
for n in range(N):
    axes[4].plot(l_history[n])
    axes[4].grid(axis='y')
    axes[4].set_ylabel("Charging from local DC-grid, Wh")
    

axes[-1].set_xlabel('Time Horizon, hour')
plt.tight_layout()
plt.savefig("Decentralised framework with data", dpi=300)
plt.show()

In [54]:
cost = np.sum(c * d_history / q[0] - p * f_history / q[0])
cost

1.5646526442656172

In [19]:
np.arange(0.01, 0.1, 0.01)

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])

In [13]:
rho_values =  np.arange(0.1, 1, 0.1)
gamma_values = np.arange(0.01, 0.1, 0.01)

for rho_trial in range(len(rho_values)):
    rho = rho_values[rho_trial]
    print(f"\n--- Starting ADMM with rho = {rho} ---\n")

    for gamma in gamma_values:
        print(f"\n--- Using gamma = {gamma} ---\n")

        # Initialize variables
        x = np.array([np.zeros(5*H) for _ in range(N)])
        x_prev = copy.deepcopy(x)
        lambdas = np.zeros(H)
        H_n_T_x = np.zeros((N, H))
        H_n_T_xi = np.zeros((N, H))

        prev_lambda_norm = None
        iteration = 0

        while True:
            iteration += 1
            final_cost = 0

            for n in range(N):
                # Define and solve optimization problem for each agent
                xi = cp.Variable(5*H)
                d = xi[:H]; f = xi[H:2*H]; l = xi[2*H:3*H]; m = xi[3*H:4*H]; L = xi[4*H:]

                start_time = L_arrival[0][n]-start
                end_time = L_depart[0][n]-start

                net_cost = cp.sum(cp.multiply(c, d) / q[0] - cp.multiply(p, f) / q[0])
                charge_penalty = cp.sum_squares(L[end_time] - L_target[0][n])
                h_n_T_x = -d + f + l - m
                admm_penalty = (rho / 2) *  cp.sum_squares(h_n_T_x + cp.sum(H_n_T_x,axis=0) - H_n_T_x[n] - q[0] - 1/rho * lambdas)
                admm_update = 11/2 * cp.sum_squares(xi - x_prev[n])

                objective = cp.Minimize(c1*net_cost + charge_penalty + admm_penalty + admm_update)

                # Constraints
                constraints = [
                    L >= L_min[n], L <= L_max[n],
                    d >= d_min[n], d <= d_max[n],
                    f >= f_min[n], f <= f_max[n],
                    l >= l_min[n], l <= l_max[n],
                    m >= m_min[n], m <= m_max[n],
                    L[start_time] == L_init[0][n] + l[start_time] - m[start_time]
                ]

                for h in range(1, H):
                    constraints += [
                        L[h] == L[h-1] + l[h] - m[h],
                        cp.abs(l[h-1] - l[h]) <= 5,
                        cp.abs(m[h-1] - m[h]) <= 5,
                        cp.abs(d[h-1] - d[h]) <= 5,
                        cp.abs(f[h-1] - f[h]) <= 5
                    ]

                for h in range(start, start + H):
                    if h >= L_depart[0,n]:
                        constraints += [l[h-start] == 0, m[h-start] == 0, d[h-start] == 0]
                    if h <= L_arrival[0,n]:
                        constraints += [l[h-start] == 0, m[h-start] == 0, d[h-start] == 0]

                prob = cp.Problem(objective, constraints)
                solution_found = False
                for solver in [cp.SCS, cp.ECOS, cp.CLARABEL, cp.OSQP, cp.GUROBI]:
                    try:
                        prob.solve(solver=solver)
                        if prob.status == "optimal":
                            solution_found = True
                            break
                    except:
                        continue

                if not solution_found:
                    print(f"Agent {n}: No solution found with any solver.")
                    continue

                x[n] = xi.value
                H_n_T_xi[n] = h_n_T_x.value
                final_cost += net_cost.value + charge_penalty.value

            # ADMM update
            H_n_T_x = copy.deepcopy(H_n_T_xi)
            residual = np.sum(H_n_T_x, axis=0) - q[0]
            lambdas -= gamma * rho * residual

            # Convergence check
            x_diff_norm = np.linalg.norm(x - x_prev)
            lambda_norm = np.linalg.norm(residual)
            print(f"Iteration {iteration}, x diff: {x_diff_norm:.6f}, lambda norm: {lambda_norm:.6f}")

            if iteration % 10 == 0:
                if prev_lambda_norm is not None and lambda_norm >= prev_lambda_norm:
                    print("Lambda norm did not decrease after 10 iterations. Trying next gamma.")
                    break  # Restart ADMM with new gamma
                prev_lambda_norm = lambda_norm

            if x_diff_norm <= epsilon:
                print("Converged!")
                break

            x_prev = copy.deepcopy(x)

        # Stop trying new gammas if already converged
        if x_diff_norm <= epsilon:
            break

    # Stop trying new rhos if already converged
    if x_diff_norm <= epsilon:
        break

print(f"Final cost: {final_cost:.2f}")




--- Starting ADMM with rho = 0.1 ---


--- Using gamma = 0.01 ---

Iteration 1, x diff: 135.622205, lambda norm: 75.839556
Iteration 2, x diff: 18.495375, lambda norm: 70.170100
Iteration 3, x diff: 17.468563, lambda norm: 67.262140
Iteration 4, x diff: 15.846342, lambda norm: 65.062759
Iteration 5, x diff: 15.188644, lambda norm: 63.315517
Iteration 6, x diff: 14.698156, lambda norm: 62.649776
Iteration 7, x diff: 14.232014, lambda norm: 62.745597
Iteration 8, x diff: 13.789100, lambda norm: 63.278062
Iteration 9, x diff: 13.315361, lambda norm: 64.038174
Iteration 10, x diff: 12.906771, lambda norm: 64.885507
Iteration 11, x diff: 12.202334, lambda norm: 64.595676
Iteration 12, x diff: 11.823674, lambda norm: 64.360508
Iteration 13, x diff: 11.224206, lambda norm: 62.985273
Iteration 14, x diff: 10.913544, lambda norm: 61.580721
Iteration 15, x diff: 10.598651, lambda norm: 60.467223
Iteration 16, x diff: 10.299079, lambda norm: 59.475446
Iteration 17, x diff: 10.031676, lambda norm